Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Proyecto Mineria de Datos</h2>
    <h1> Predecir un accidente cerebrovascular </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Primer Semestre 2023<br>    
        Fecha de entrega: Martes 13 de Junio
    </p>
    <p>Integrantes: Lucas Aguilera, Claudio Bórquez, Josefa Fernández
    </p>
    <br>
</center>

<br>

---

Se importan las librerias ausar en este proyecto:

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1-Analisis de Datos y Preparación

## Parte 1: Carga de datos
- Se descarga el conjunto de datos del enlace proporcionado.

Durante este proyecto se usan datos clínicos, centrados en accidentes cerebrovasculares, obtendios de Kaggle:

https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset



In [3]:
dframe = pd.read_csv("healthcare-dataset-stroke-data.csv", encoding = "ISO-8859-1")

## Parte 2: Exploración de datos
- Se exploran los datos para comprender la estrucutura, caracteristicas y posibles problemas.


In [10]:
dframe.shape

(5110, 12)

In [12]:
dframe.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


- Los datos contienen 5110 observaciones con 12 atributos.
- Se observa valores nulos en la columna bmi



El dataset se utiliza para predecir si es probable que un paciente sufra un accidente cerebrovascular en función de los parámetros de entrada como el sexo, la edad, diversas enfermedades y el tabaquismo. Cada fila de los datos proporciona información relevante sobre el paciente.

El dataset, que se usará  para construir un modelo para predecir. Los atributos que contiene son:
- id: identificador unico
- gender: "Male", "Female" or "Other" ("Masculino", "Femenino" u "Otro")
- age: edad del paciente
- hypertension: 0 si el paciente no tiene hipertension, 1 si el paciente tiene hipertension
- heart_disease: 0 si el paciente no tiene ninguna enfermedad al corazón, 1 si el paciente tiene alguna enfermedad al corazón 
- ever_married: "No" or "Yes" ("No" o "Si")
- work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed" ("niños", "Govt_jov", "nunca_ha_trabajo", "Privado" o "Independiente")
- Residence_type: "Rural" or "Urban" ("Rural" o "Urbano")
- avg_glucose_level: el promedio del nivel de glucosa en la sangre
- bmi: índice de masa corporal
- smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"* ("anteriormente fumó", "nunca fumó", "fuma"o "Desconocido"*)
- stroke: 1 si el paciente tuvo un accidente cerebrovascular o 0 si no

*Nota: "Desconocido" en smoking_status significa que la información no está disponible para este paciente

Toda esta informacion fue sacada de kaggle.

## Parte 3: Preprocesamiento (15 ptos.)
- Se realiza una limpieza de datos para manejar valores faltantes, duplicados u otros errores.
- Realizar una transformación de datos si es necesario, como codificación de variables categóricas.

Se preprocesan los datos, por lo que consideraros si es necesario hacer entre otras, las siguientes cosas:
- Remover columnas
- Normalizar variables
- Manejo de valores nulos

*VER S HACEMOS ESTO: Para manejar los datos categoricos tienes que usar One Hot Encoding.

In [5]:
dframe.head(5) #verifico los 1eros 5 registros del dframe

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


#### Veo valores nulos 

In [8]:
dframe[dframe.isnull().any(axis=1)].size #numero total de valores nulos del dframe el rsultado dice 0

2412

In [9]:
dframe.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

### Remover columnas
Saco columnas que no aportan informacion relevante para el modelo  en predecir la supervivencia de los pasajeros y cabin porque tiene muchos datos nulos.

In [11]:
dframe.drop([''], axis=1, inplace=True)

### One Hot Encoding
Dado que hay datos categoricos se debe codificar usando esto. Las columnas ''  son categoricas, por lo que uso este metodo para convertirlas en variables numericas.

### Normalizar variables


# 2- Preprocesamiento y selección de caracteristicas

## Parte 1: Análisis exploratorio
- Realizar un análisis exploratorio de datos para obtener información relevante.


## Parte 2: Aplicar tecnicas de preprocesamiento
- Aplicar técnicas de preprocesamiento, como escalamiento o normalización, según sea necesario.


## Parte 3: Selección de caracteristicas
- Seleccionar las características más relevantes para la predicción del accidente cerebrovascular.


# 3-Implementación de algoritmos de minería de datos

## Parte 1: MLP 
- Utilizar el algoritmo MLP (Perceptrón Multicapa) para construir un modelo de predicción.


## Parte 2: GMM y Regresion Logistica 
- Implementar Regresión Logística y GMM (Modelos de Mezcla Gaussiana) como parte de un pipeline extendido. Para construir un modelo de predicción.


# 4-Evaluación del modelo

## Parte 1: División de datos en train y test
- Dividir los datos en conjuntos de entrenamiento y prueba.

## Parte 2: Entrenar los modelos
- Entrenar los modelos con los datos de entrenamiento y evaluar su rendimiento con los datos de prueba.

## Parte 3: Metricas de evaluación
- Utilizar métricas de evaluación adecuadas, como precisión, sensibilidad, especificidad y F1-score.

# 5-Análisis de resultados

## Parte 1: Interpretación de resultados
- Interpretar y analizar los resultados obtenidos de los diferentes modelos.

## Parte 2: Fortalezas y debilidades de cada modelo
- Identificar fortalezas y debilidades de cada algoritmo en términos de precisión y eficiencia.

## Parte 3: Por mejorar
- Realizar ajustes o mejoras en el modelo según sea necesario.